<a href="https://colab.research.google.com/github/muslimkhudoyberdiev/voice_assistant/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [3]:
#importing the dataset
with open('content.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
print(data)

                                               inputs           tags
0                                               Salom       greeting
1                                     Assalom aleykum       greeting
2                                         Yaxshimisiz       greeting
3                                 menda savol bor edi       greeting
4                                        Kimdir bormi       greeting
5                                               hello       greeting
6                                                helo       greeting
7                                                 hey       greeting
8                                              salyut       greeting
9                                            qaleysan       greeting
10                                                  ?       greeting
11                                             kimsiz       greeting
12                                           salomlar       greeting
13                                

In [4]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [6]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)


13
number of unique words :  79
output length:  7


In [7]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
2/2 [==============================] - 10s 16ms/step - loss: 1.9467 - accuracy: 0.0833
Epoch 2/200
2/2 [==============================] - 0s 39ms/step - loss: 1.9427 - accuracy: 0.1250
Epoch 3/200
2/2 [==============================] - 0s 21ms/step - loss: 1.9385 - accuracy: 0.3125
Epoch 4/200
2/2 [==============================] - 0s 33ms/step - loss: 1.9338 - accuracy: 0.2917
Epoch 5/200
2/2 [==============================] - 0s 19ms/step - loss: 1.9294 - accuracy: 0.3125
Epoch 6/200
2/2 [==============================] - 0s 24ms/step - loss: 1.9238 - accuracy: 0.3125
Epoch 7/200
2/2 [==============================] - 0s 24ms/step - loss: 1.9185 - accuracy: 0.2917
Epoch 8/200
2/2 [==============================] - 0s 19ms/step - loss: 1.9112 - accuracy: 0.2708
Epoch 9/200
2/2 [==============================] - 0s 25ms/step - loss: 1.9048 - accuracy: 0.2708
Epoch 10/200
2/2 [==============================] - 0s 32ms/step - loss: 1.8972 - accuracy: 0.2708
Epoch 11/200
2/2 [

In [9]:
#chatting
import random
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Going Merry : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You : salyut
Going Merry :  Assalom aleykum yaxshimisiz. Men Demo telecom kompaniyasining mobil operatoriman, sizga qanday yordam bera olaman?
You : o'chir ovozingni
Going Merry :  Assalom aleykum yaxshimisiz. Men Demo telecom kompaniyasining mobil operatoriman, sizga qanday yordam bera olaman?
You : asdwfawf
Going Merry :  Assalom aleykum yaxshimisiz. Men Demo telecom kompaniyasining mobil operatoriman, sizga qanday yordam bera olaman?
You : yordam
Going Merry :  Aha sizga sizni tushundim, qanday savolingiz bor
You : telefonni ko'tar
Going Merry :  Assalom aleykum yaxshimisiz. Men Demo telecom kompaniyasining mobil operatoriman, sizga qanday yordam bera olaman?
You : manzil
Going Merry :  Assalom aleykum yaxshimisiz. Men Demo telecom kompaniyasining mobil operatoriman, sizga qanday yordam bera olaman?
You : manzilingni ber
Going Merry :  Assalom aleykum yaxshimisiz. Men Demo telecom kompaniyasining mobil operatoriman, sizga qanday yordam bera olaman?
You : savol
Going Merry :  Aha siz

KeyboardInterrupt: ignored